<a href="https://colab.research.google.com/github/Valgueiro/dataScience_project/blob/master/Projeto_1_dataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisandos os dados da temporada 2019/20 da NFL

Utilizarei os dados de https://www.kaggle.com/dvc415/nfl-2019-weekly-player-stats . Nele podemos encontrar estatísticas sobre diversos jogadores de 3 posiçoes do futebol americano:

* Quaterbacks (qbs) - A mente por trás do ataque, são os responsáveis por liderar e coordenar o ataque

* Running Backs (rbs) - corredores da equipe, geralmente recebem a bola diretamente do Quarteback e correm para dentro da defesa.

* Wide receivers (wrs) - também são corredores, mas geralmente correm pra dentro da defesa adversária e recebem a bola do QB através de um passe.

## Pré-processando os dados

### Coletando dados



In [2]:
import numpy as np
import pandas as pd

In [3]:
qbs = pd.read_csv("https://raw.githubusercontent.com/Valgueiro/dataScience_project/master/data/nfl_2019_qb_weekly_stats.csv")
rbs = pd.read_csv("https://raw.githubusercontent.com/Valgueiro/dataScience_project/master/data/nfl_2019_rb_weekly_stats.csv")
wrs = pd.read_csv("https://raw.githubusercontent.com/Valgueiro/dataScience_project/master/data/nfl_2019_wr_weekly_stats.csv")

### Filtrando e reorganizando os dados de Qbs

In [4]:
qbs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           611 non-null    int64  
 1   Player         611 non-null    object 
 2   Position       611 non-null    object 
 3   Age            611 non-null    float64
 4   Date           611 non-null    object 
 5   League         611 non-null    object 
 6   Team           611 non-null    object 
 7   Opp            611 non-null    object 
 8   Result         611 non-null    object 
 9   Game           611 non-null    int64  
 10  Week           611 non-null    int64  
 11  Day            611 non-null    object 
 12  Cmp            611 non-null    int64  
 13  Pass_Att       611 non-null    int64  
 14  Cmp_Per        611 non-null    float64
 15  Pass_Yards     611 non-null    int64  
 16  Pass_TD        611 non-null    int64  
 17  Pass_Int       611 non-null    int64  
 18  QB_Rate   

Logo de cara podemos ver que não temos dados ausentes, mas alguns tipos devem ser estudados e melhorados.

#### Verificando a necessidade da coluna "Position"

In [5]:
qbs["Position"].unique()

array(['QB', 'WR', 'RB', 'K', 'P', 'FB/TE'], dtype=object)

In [6]:
qbs[qbs["Position"] == 'WR'][["Player", "Pass_Att"]]

,Player,Pass_Att
14,Julian Edelman,1
43,Dante Pettis,1
142,Odell Beckham Jr.,1
146,DeAndre Hopkins,1
151,Odell Beckham Jr.,1
248,Zach Pascal,1
338,Randall Cobb,1
343,Julian Edelman,1
348,Courtland Sutton,1
411,Steven Sims,1


Como podemos ver, o dataset conta também com jogadores de outra posição que jogaram de QB em algum momento, como por exemplo os Wide Receivers que vemos nesta ultima tabela. Com as duas colunas mostradas podemos ver que todos eles fizeram apenas 2 passes no máximo. Geralmente essas jogadas são fora do comum, são chamadas de "trick play".

#### Verificando o tipo da coluna "Age"

In [7]:
qbs["Age"].describe()

count    611.000000
mean      28.154298
std        5.364599
min       22.032000
25%       24.046500
50%       26.308000
75%       31.219000
max       42.148000
Name: Age, dtype: float64

In [8]:
qbs[qbs["Player"] == "Kirk Cousins"][["Player","Age","Date"]]

,Player,Age,Date
4,Kirk Cousins,31.020,2019-09-08
72,Kirk Cousins,31.027,2019-09-15
86,Kirk Cousins,31.034,2019-09-22
126,Kirk Cousins,31.041,2019-09-29
149,Kirk Cousins,31.048,2019-10-06
186,Kirk Cousins,31.055,2019-10-13
219,Kirk Cousins,31.062,2019-10-20
259,Kirk Cousins,31.066,2019-10-24
298,Kirk Cousins,31.076,2019-11-03
318,Kirk Cousins,31.083,2019-11-10


Como podemos ver, apesar de idade normalmente ser tratada com inteiro, aqui é mostrada a idade inclusive incluindo a quantidade de meses e dias.

#### Transformando o tipo da coluna Date

Com o describe, vimos que a coluna Date está com o tipo Object. Devemos mudá-la para o tipo correto de datas.

In [9]:
qbs["Date"].head()

0    2019-09-08
1    2019-09-08
2    2019-09-08
3    2019-09-08
4    2019-09-08
Name: Date, dtype: object

In [10]:
qbs["Date"] = pd.to_datetime(qbs["Date"])
qbs["Date"].head()

0   2019-09-08
1   2019-09-08
2   2019-09-08
3   2019-09-08
4   2019-09-08
Name: Date, dtype: datetime64[ns]

#### Verificando a necessidade da coluna League

In [11]:
qbs["League"].unique()

array(['NFL'], dtype=object)

Como podemos ver todas as linhas tem o mesmo valor para esta coluna. Como já é sabido que estamos no contexto da NFL, vou retirá-la do dataset.

In [12]:
qbs.drop(columns=['League'])

,Rank,Player,Position,Age,Date,Team,Opp,Result,Game,Week,Day,Cmp,Pass_Att,Cmp_Per,Pass_Yards,Pass_TD,Pass_Int,QB_Rate,Sacked,Yds_Lost,Yards_Att,Adj_Yards_Att
0,1,Lamar Jackson,QB,22.244,2019-09-08,BAL,MIA,W 59-10,1,1,Sun,17,20,85.00,324,5,0,158.3,1,1,16.20,16.20
1,2,Dak Prescott,QB,26.041,2019-09-08,DAL,NYG,W 35-17,1,1,Sun,25,32,78.13,405,4,0,158.3,0,0,12.66,12.66
2,3,Robert Griffin III,QB,29.208,2019-09-08,BAL,MIA,W 59-10,1,1,Sun,6,6,100.00,55,1,0,144.4,0,0,9.17,9.17
3,4,Patrick Mahomes,QB,23.356,2019-09-08,KAN,JAX,W 40-26,1,1,Sun,25,33,75.76,378,3,0,143.2,0,0,11.45,11.45
4,5,Kirk Cousins,QB,31.020,2019-09-08,MIN,ATL,W 28-12,1,1,Sun,8,10,80.00,98,1,0,140.8,1,1,9.80,9.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,36,Robert Griffin III,QB,29.320,2019-12-29,BAL,PIT,W 28-10,16,17,Sun,11,21,52.38,96,0,1,44.9,3,15,4.57,4.57
607,37,David Blough,QB,24.151,2019-12-29,DET,GNB,L 20-23,16,17,Sun,12,29,41.38,122,0,1,39.7,1,7,4.21,4.21
608,38,Matt Haack,P,25.187,2019-12-29,MIA,NWE,W 27-24,16,17,Sun,0,1,0.00,0,0,0,39.6,0,0,0.00,0.00
609,39,Sean Mannion,QB,27.248,2019-12-29,MIN,CHI,L 19-21,16,17,Sun,12,21,57.14,126,0,2,35.1,0,0,6.00,6.00


#### Separar dados da coluna Result

In [13]:
qbs["Result"].head()

0    W 59-10
1    W 35-17
2    W 59-10
3    W 40-26
4    W 28-12
Name: Result, dtype: object

A coluna result contem 3 informaçoes: O resultado final do jogo para o jogador, a quantidade de pontos feitos e a quantidade de pontos recebidos. Vamos separar esses dados em 3 colunas distintas

In [14]:
import re
parsed_results = qbs["Result"].map(lambda result: re.findall('(.+) (\d+)-(\d+)', result)[0])
qbs['Game_result'] = parsed_results.map(lambda parsed: parsed[0])
qbs['Points_scored'] = parsed_results.map(lambda parsed: parsed[1])
qbs['Points_Taken'] = parsed_results.map(lambda parsed: parsed[2])

qbs[['Game_result', 'Points_scored', 'Points_Taken']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Game_result    611 non-null    object
 1   Points_scored  611 non-null    object
 2   Points_Taken   611 non-null    object
dtypes: object(3)
memory usage: 14.4+ KB


Após dividir as informaçoes, vamos mudar o tipo para inteiro e apagar a coluna que foi dividida.

In [15]:
qbs['Points_scored'] = pd.to_numeric(qbs['Points_scored'])
qbs['Points_Taken'] = pd.to_numeric(qbs['Points_Taken'])
qbs[['Game_result', 'Points_scored', 'Points_Taken']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Game_result    611 non-null    object
 1   Points_scored  611 non-null    int64 
 2   Points_Taken   611 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 14.4+ KB


In [23]:
qbs = qbs.drop(columns=['Result'])

### Resultado final


In [24]:
qbs.head()

,Rank,Player,Position,Age,Date,League,Team,Opp,Game,Week,Day,Cmp,Pass_Att,Cmp_Per,Pass_Yards,Pass_TD,Pass_Int,QB_Rate,Sacked,Yds_Lost,Yards_Att,Adj_Yards_Att,Game_result,Points_scored,Points_Taken
0,1,Lamar Jackson,QB,22.244,2019-09-08,NFL,BAL,MIA,1,1,Sun,17,20,85.00,324,5,0,158.3,1,1,16.20,16.20,W,59,10
1,2,Dak Prescott,QB,26.041,2019-09-08,NFL,DAL,NYG,1,1,Sun,25,32,78.13,405,4,0,158.3,0,0,12.66,12.66,W,35,17
2,3,Robert Griffin III,QB,29.208,2019-09-08,NFL,BAL,MIA,1,1,Sun,6,6,100.00,55,1,0,144.4,0,0,9.17,9.17,W,59,10
3,4,Patrick Mahomes,QB,23.356,2019-09-08,NFL,KAN,JAX,1,1,Sun,25,33,75.76,378,3,0,143.2,0,0,11.45,11.45,W,40,26
4,5,Kirk Cousins,QB,31.020,2019-09-08,NFL,MIN,ATL,1,1,Sun,8,10,80.00,98,1,0,140.8,1,1,9.80,9.80,W,28,12


In [25]:
qbs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Rank           611 non-null    int64         
 1   Player         611 non-null    object        
 2   Position       611 non-null    object        
 3   Age            611 non-null    float64       
 4   Date           611 non-null    datetime64[ns]
 5   League         611 non-null    object        
 6   Team           611 non-null    object        
 7   Opp            611 non-null    object        
 8   Game           611 non-null    int64         
 9   Week           611 non-null    int64         
 10  Day            611 non-null    object        
 11  Cmp            611 non-null    int64         
 12  Pass_Att       611 non-null    int64         
 13  Cmp_Per        611 non-null    float64       
 14  Pass_Yards     611 non-null    int64         
 15  Pass_TD        611 non-

## Analisando os dados

### Os mais bem rankeados

Primeiramente vamos ver quais foram os qbs mais bem rankeados na temporada. Para isso, vamos agrupar os dados por jogador.

In [66]:
all_players_qb_stats = qbs.groupby(['Player', 'Position'])
all_players_qb_stats.head()

,Rank,Player,Position,Age,Date,League,Team,Opp,Game,Week,Day,Cmp,Pass_Att,Cmp_Per,Pass_Yards,Pass_TD,Pass_Int,QB_Rate,Sacked,Yds_Lost,Yards_Att,Adj_Yards_Att,Game_result,Points_scored,Points_Taken
0,1,Lamar Jackson,QB,22.244,2019-09-08,NFL,BAL,MIA,1,1,Sun,17,20,85.00,324,5,0,158.3,1,1,16.20,16.20,W,59,10
1,2,Dak Prescott,QB,26.041,2019-09-08,NFL,DAL,NYG,1,1,Sun,25,32,78.13,405,4,0,158.3,0,0,12.66,12.66,W,35,17
2,3,Robert Griffin III,QB,29.208,2019-09-08,NFL,BAL,MIA,1,1,Sun,6,6,100.00,55,1,0,144.4,0,0,9.17,9.17,W,59,10
3,4,Patrick Mahomes,QB,23.356,2019-09-08,NFL,KAN,JAX,1,1,Sun,25,33,75.76,378,3,0,143.2,0,0,11.45,11.45,W,40,26
4,5,Kirk Cousins,QB,31.020,2019-09-08,NFL,MIN,ATL,1,1,Sun,8,10,80.00,98,1,0,140.8,1,1,9.80,9.80,W,28,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,34,Matt Barkley,QB,29.112,2019-12-29,NFL,BUF,NYJ,16,17,Sun,18,35,51.43,232,0,2,48.8,1,1,6.63,6.63,L,6,13
607,37,David Blough,QB,24.151,2019-12-29,NFL,DET,GNB,16,17,Sun,12,29,41.38,122,0,1,39.7,1,7,4.21,4.21,L,20,23
608,38,Matt Haack,P,25.187,2019-12-29,NFL,MIA,NWE,16,17,Sun,0,1,0.00,0,0,0,39.6,0,0,0.00,0.00,W,27,24
609,39,Sean Mannion,QB,27.248,2019-12-29,NFL,MIN,CHI,16,17,Sun,12,21,57.14,126,0,2,35.1,0,0,6.00,6.00,L,19,21


Primeiramente, podemos observar a coluna de QB_Rate, que trás o Passer Rating do jogador em cada uma das partidas que ele jogou. 

O cáculo deste número é definido pela liga e leva em comparaçao atributos como o número de passes tentados, de passes completos, touchdowns, etc. O interessante desta métrica é que seu valor máximo é de **158,3**, e seu valor mínimo,** 0**.

Para mais informações sobre como é feito seu cálculo https://pt.wikipedia.org/wiki/Passer_rating


In [67]:
all_players_qb_stats['Player', 'Position', 'QB_Rate'].mean().sort_values(['QB_Rate'],ascending=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,,QB_Rate
Player,Position,
Danny Amendola,WR,158.30
Emmanuel Sanders,WR,158.30
John Brown,WR,158.30
Julian Edelman,WR,138.50
Matt Schaub,QB,120.70
...,...,...
Jarrett Stidham,QB,38.55
Sean Mannion,QB,35.10
Will Grier,QB,23.00


Como podemos ver nos valores maiores e menores mostrados, todos os jogadores com o rate máximo são de posições diferentes de QB. Isso se deve marjoritariamente ao fato de que jogadores de outras posições podem até jogar como QB em jogadas "engraçadinhas", mas essas são poucas durante uma temporada inteira. Entao, vamos retirar esses outros jogadores.

In [69]:
qb_players = qbs[qbs['Position'] == 'QB'].groupby(['Player', 'Position'])
qb_players.head()

,Rank,Player,Position,Age,Date,League,Team,Opp,Game,Week,Day,Cmp,Pass_Att,Cmp_Per,Pass_Yards,Pass_TD,Pass_Int,QB_Rate,Sacked,Yds_Lost,Yards_Att,Adj_Yards_Att,Game_result,Points_scored,Points_Taken
0,1,Lamar Jackson,QB,22.244,2019-09-08,NFL,BAL,MIA,1,1,Sun,17,20,85.00,324,5,0,158.3,1,1,16.20,16.20,W,59,10
1,2,Dak Prescott,QB,26.041,2019-09-08,NFL,DAL,NYG,1,1,Sun,25,32,78.13,405,4,0,158.3,0,0,12.66,12.66,W,35,17
2,3,Robert Griffin III,QB,29.208,2019-09-08,NFL,BAL,MIA,1,1,Sun,6,6,100.00,55,1,0,144.4,0,0,9.17,9.17,W,59,10
3,4,Patrick Mahomes,QB,23.356,2019-09-08,NFL,KAN,JAX,1,1,Sun,25,33,75.76,378,3,0,143.2,0,0,11.45,11.45,W,40,26
4,5,Kirk Cousins,QB,31.020,2019-09-08,NFL,MIN,ATL,1,1,Sun,8,10,80.00,98,1,0,140.8,1,1,9.80,9.80,W,28,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,31,A.J. McCarron,QB,29.107,2019-12-29,NFL,HOU,TEN,16,17,Sun,21,36,58.33,225,0,1,65.2,4,33,6.25,6.25,L,14,35
604,34,Matt Barkley,QB,29.112,2019-12-29,NFL,BUF,NYJ,16,17,Sun,18,35,51.43,232,0,2,48.8,1,1,6.63,6.63,L,6,13
607,37,David Blough,QB,24.151,2019-12-29,NFL,DET,GNB,16,17,Sun,12,29,41.38,122,0,1,39.7,1,7,4.21,4.21,L,20,23
609,39,Sean Mannion,QB,27.248,2019-12-29,NFL,MIN,CHI,16,17,Sun,12,21,57.14,126,0,2,35.1,0,0,6.00,6.00,L,19,21


Agora vamos analisar os ratings apenas dos Quartebacks presentes no dataset

In [74]:
sorted_by_rate = qb_players['Player', 'Position', 'QB_Rate'].mean().sort_values(['QB_Rate'],ascending=False)
sorted_by_rate.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,,QB_Rate
Player,Position,
Matt Schaub,QB,120.700000
Ryan Tannehill,QB,118.927273
Drew Brees,QB,109.872727
Lamar Jackson,QB,108.460000
Kirk Cousins,QB,108.066667
Russell Wilson,QB,107.793750
Matthew Stafford,QB,105.100000
Patrick Mahomes,QB,104.842857
Mike Glennon,QB,104.700000


Vemos outro ponto estranho: **Matt Schaub** é o Quarterback Reserva de Texas e jogou apenas 1 jogo na temporada 2019. Contudo, aparentemente, este foi o jogo da sua vida! Vamos analisar melhor os dados para ver como podemos limpar casos como esses:

In [77]:
import plotly.express as px

fig = px.bar(sorted_by_rate, x="QB_Rate", y="QB_Rate")
fig.show()